# Carvana U-Net Angles

## Imports

In [1]:
from keras.layers.advanced_activations import PReLU
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
import keras.backend as K
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from optimizers.AdamAccumulate import AdamAccumulate
from models.u_net import UNet
from models.u_net_aux import UNet_Aux
from models.u_net_heng import UNet_Heng
from utilities.submit import generate_submit
from utilities import utils_masks as utils
from utilities.losses import weighted_bce_dice_loss, dice_value

%load_ext autoreload
%autoreload 2
%matplotlib inline

Using TensorFlow backend.


## Preparing Data

In [2]:
utils.set_results_reproducible()
input_size = 128
train_path = "inputs/train/{}.jpg" 
train_mask_path = "inputs/train_masks/{}_mask.gif"

df = pd.read_csv('inputs/train_masks.csv')
all_ids = df['img'].map(lambda s: s.split('.')[0])
all_ids_train_split, all_ids_valid_split = train_test_split(all_ids, test_size=0.2, random_state=42)

groups = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16']
ids_train_splits = {}
ids_valid_splits = {}

for group in groups:
    df_group = df[(df.img.str.match('^.*_(' + group + ').jpg$'))]
    ids_group = df_group['img'].map(lambda s: s.split('.')[0])
    ids_train_split = pd.Series(list(set(all_ids_train_split).intersection(set(ids_group))))
    ids_valid_split = pd.Series(list(set(all_ids_valid_split).intersection(set(ids_group))))
    ids_train_splits[group] = ids_train_split
    ids_valid_splits[group] = ids_valid_split
    print('group {0}:   #Training = {1}   #Validation = {2}'.format(group, len(ids_train_split), len(ids_valid_split)))

#bboxes = None
bbox_file_path = 'inputs/train_bbox.csv'
bboxes = utils.get_bboxes(bbox_file_path)

def train_generator(batch_size, group, outputs=None):
    return utils.train_generator(train_path, train_mask_path, ids_train_splits[group], 
                                 input_size, batch_size, bboxes, outputs=outputs,
                                 augmentations=['HUE_SATURATION', 'SHIFT_SCALE'])

def valid_generator(batch_size, group, outputs=None):
    return utils.valid_generator(train_path, train_mask_path, ids_valid_splits[group],
                                 input_size, batch_size, bboxes, outputs=outputs)

group 01:   #Training = 259   #Validation = 59
group 02:   #Training = 260   #Validation = 58
group 03:   #Training = 259   #Validation = 59
group 04:   #Training = 257   #Validation = 61
group 05:   #Training = 256   #Validation = 62
group 06:   #Training = 249   #Validation = 69
group 07:   #Training = 259   #Validation = 59
group 08:   #Training = 257   #Validation = 61
group 09:   #Training = 247   #Validation = 71
group 10:   #Training = 254   #Validation = 64
group 11:   #Training = 248   #Validation = 70
group 12:   #Training = 259   #Validation = 59
group 13:   #Training = 252   #Validation = 66
group 14:   #Training = 254   #Validation = 64
group 15:   #Training = 261   #Validation = 57
group 16:   #Training = 239   #Validation = 79


## Create Model

In [3]:
#U-Net-Aux:
#model = UNet_Aux((input_size, input_size, 3), filters=64, depth=4, dropout_base_only=False, dropout=0,
#                 activation=lambda x: PReLU()(x), init='he_uniform', auxiliaries=[False, True, True, False])
#outputs = {'aux_out1':2**-1, 'aux_out2':2**-2, 'main_out':1}
#weights = {'aux_out1':0.2, 'aux_out2':0.05, 'main_out':1.}
#model.compile(optimizer=AdamAccumulate(accum_iters=4), 
#              loss=weighted_bce_dice_loss, metrics=[dice_value], loss_weights=weights)

#U-Net:
#model = UNet((input_size, input_size, 3), filters=64, depth=4, dropout_base_only=False, dropout=0,
#             activation=lambda x: PReLU()(x), init='he_uniform')
#model.compile(optimizer=AdamAccumulate(accum_iters=4), loss=weighted_bce_dice_loss, metrics=[dice_value])

## Fit Models

In [4]:
epochs = 150
batch_size = 1

for group in groups:
    #model = UNet_Heng((input_size, input_size, 3))
    model = UNet((input_size, input_size, 3), filters=64, depth=4, dropout_base_only=False, dropout=0,
             activation=lambda x: PReLU()(x), init='he_uniform')
    model.compile(optimizer=AdamAccumulate(accum_iters=32),
                  loss=weighted_bce_dice_loss, metrics=[dice_value])
    run_name = utils.get_run_name('weights/{}.hdf5', 'unet-({})'.format(group))
    weights_path = 'weights/{}.hdf5'.format(run_name)

    callbacks = [EarlyStopping(monitor='val_dice_value',
                               patience=8,
                               verbose=1,
                               min_delta=1e-4,
                               mode='max'),
                 ReduceLROnPlateau(monitor='val_dice_value',
                                   factor=0.1,
                                   patience=4,
                                   verbose=1,
                                   epsilon=1e-4,
                                   mode='max'),
                 ModelCheckpoint(monitor='val_dice_value',
                                 filepath=weights_path,
                                 save_best_only=True,
                                 save_weights_only=True,
                                 mode='max'),
                 TensorBoard(log_dir='logs/{}'.format(run_name), batch_size=batch_size)]

    model.load_weights('weights/unet-2017-09-22-2306.hdf5')
    K.set_value(model.optimizer.lr, 1e-4)

    print('Starting run "{}"'.format(run_name))
    model.fit_generator(generator=train_generator(batch_size, group),
                        steps_per_epoch=np.ceil(float(len(ids_train_splits[group])) / float(batch_size)),
                        epochs=epochs,
                        verbose=1,
                        callbacks=callbacks,
                        validation_data=valid_generator(batch_size, group),
                        validation_steps=np.ceil(float(len(ids_valid_splits[group])) / float(batch_size)))
    print()
    K.clear_session()
    sess = tf.Session()
    K.set_session(sess)

Starting run "unet-(01)-2017-09-25-0621"
Epoch 1/150
259/259 [==============================] - 29s - loss: 0.4318 - dice_value: 0.9931 - val_loss: 0.4140 - val_dice_value: 0.9958
Epoch 2/150
259/259 [==============================] - 25s - loss: 0.4277 - dice_value: 0.9935 - val_loss: 0.4142 - val_dice_value: 0.9957
Epoch 3/150
259/259 [==============================] - 25s - loss: 0.4283 - dice_value: 0.9941 - val_loss: 0.4141 - val_dice_value: 0.9957
Epoch 4/150
259/259 [==============================] - 25s - loss: 0.4292 - dice_value: 0.9940 - val_loss: 0.4141 - val_dice_value: 0.9957
Epoch 5/150
259/259 [==============================] - 25s - loss: 0.4273 - dice_value: 0.9941 - val_loss: 0.4144 - val_dice_value: 0.9957
Epoch 6/150
258/259 [============================>.] - ETA: 0s - loss: 0.4303 - dice_value: 0.9937
Epoch 00005: reducing learning rate to 9.999999747378752e-06.
259/259 [==============================] - 25s - loss: 0.4303 - dice_value: 0.9937 - val_loss: 0.4141 -

256/256 [==============================] - 24s - loss: 0.4829 - dice_value: 0.9933 - val_loss: 0.4718 - val_dice_value: 0.9949
Epoch 8/150
256/256 [==============================] - 24s - loss: 0.4797 - dice_value: 0.9934 - val_loss: 0.4718 - val_dice_value: 0.9949
Epoch 9/150
256/256 [==============================] - 24s - loss: 0.4806 - dice_value: 0.9935 - val_loss: 0.4718 - val_dice_value: 0.9949
Epoch 10/150
255/256 [============================>.] - ETA: 0s - loss: 0.4808 - dice_value: 0.9935
Epoch 00009: reducing learning rate to 9.999999747378752e-07.
256/256 [==============================] - 24s - loss: 0.4808 - dice_value: 0.9935 - val_loss: 0.4718 - val_dice_value: 0.9949
Epoch 00009: early stopping

Starting run "unet-(06)-2017-09-25-0645"
Epoch 1/150
249/249 [==============================] - 26s - loss: 0.4917 - dice_value: 0.9931 - val_loss: 0.4797 - val_dice_value: 0.9956
Epoch 2/150
249/249 [==============================] - 23s - loss: 0.4887 - dice_value: 0.9935 - 

254/254 [==============================] - 25s - loss: 0.4567 - dice_value: 0.9942 - val_loss: 0.4493 - val_dice_value: 0.9958
Epoch 4/150
254/254 [==============================] - 24s - loss: 0.4547 - dice_value: 0.9940 - val_loss: 0.4493 - val_dice_value: 0.9958
Epoch 5/150
254/254 [==============================] - 25s - loss: 0.4547 - dice_value: 0.9942 - val_loss: 0.4490 - val_dice_value: 0.9959
Epoch 6/150
253/254 [============================>.] - ETA: 0s - loss: 0.4539 - dice_value: 0.9946
Epoch 00005: reducing learning rate to 9.999999747378752e-06.
254/254 [==============================] - 24s - loss: 0.4538 - dice_value: 0.9946 - val_loss: 0.4491 - val_dice_value: 0.9959
Epoch 7/150
254/254 [==============================] - 25s - loss: 0.4531 - dice_value: 0.9946 - val_loss: 0.4490 - val_dice_value: 0.9959
Epoch 8/150
254/254 [==============================] - 25s - loss: 0.4519 - dice_value: 0.9947 - val_loss: 0.4489 - val_dice_value: 0.9959
Epoch 9/150
254/254 [========

254/254 [==============================] - 25s - loss: 0.4920 - dice_value: 0.9940 - val_loss: 0.4845 - val_dice_value: 0.9954
Epoch 11/150
253/254 [============================>.] - ETA: 0s - loss: 0.4921 - dice_value: 0.9939
Epoch 00010: reducing learning rate to 9.999999747378752e-07.
254/254 [==============================] - 24s - loss: 0.4920 - dice_value: 0.9939 - val_loss: 0.4845 - val_dice_value: 0.9954
Epoch 00010: early stopping

Starting run "unet-(15)-2017-09-25-0727"
Epoch 1/150
261/261 [==============================] - 28s - loss: 0.4874 - dice_value: 0.9940 - val_loss: 0.4802 - val_dice_value: 0.9956
Epoch 2/150
261/261 [==============================] - 26s - loss: 0.4870 - dice_value: 0.9943 - val_loss: 0.4800 - val_dice_value: 0.9957
Epoch 3/150
261/261 [==============================] - 25s - loss: 0.4879 - dice_value: 0.9944 - val_loss: 0.4801 - val_dice_value: 0.9957
Epoch 4/150
261/261 [==============================] - 25s - loss: 0.4881 - dice_value: 0.9942 - 

## Mask Prediction

In [ ]:
from PIL import Image
import cv2
from glob import glob

In [ ]:
model.load_weights('weights/unet-2017-09-02-1809.hdf5')

In [ ]:
imgs_names = glob('inputs/train/*.jpg')

In [ ]:
num = 1
input_size = 1024
pred_mask_path = "outputs/pred_train_masks/{}_mask.gif"
for i in range(len(imgs_names)):
    im = cv2.imread(imgs_names[i])
    im = cv2.resize(im, (input_size, input_size), interpolation=cv2.INTER_LINEAR)
    im = np.expand_dims(im, axis=0)
    im = np.array(im, np.float32) / 255
    #print im.shape
    pred_mask = model.predict(im, batch_size=1, verbose=2)
    #print pred_mask.shape
    pred_mask = np.squeeze(pred_mask, axis=[0, 3])
    #print pred_mask.shape
    pred_mask = cv2.resize(pred_mask, (1918, 1280), interpolation=cv2.INTER_LINEAR)
    #print pred_mask.shape
    pred_mask = (pred_mask > 0.95)
    pred_mask = Image.fromarray((pred_mask * 255).astype(np.uint8), mode='L')
    im_name = imgs_names[i].split('/')[-1]
    im_id = im_name.split('.')[0]
    pred_mask.save(pred_mask_path.format(im_id))
    
    if num%500==0:
        print '{0}/{1}'.format(num, len(imgs_names))
    num += 1

In [ ]:
#val_imgs, _ = next(valid_generator(len(ids_valid_split)))
train_imgs, _ = next(train_generator(len(ids_train_split[4000:])))

In [ ]:
#print val_imgs.shape
print train_imgs.shape

In [ ]:
#val_pred_masks = model.predict(val_imgs, batch_size=16, verbose=1)
train_pred_masks = model.predict(train_imgs, batch_size=16, verbose=1)

In [ ]:
preds = np.squeeze(train_pred_masks, axis=3)
np.shape(preds)

In [ ]:
pred_mask_path = "outputs/valid_masks/{}_mask.gif"

In [ ]:
for i in range(train_imgs.shape[0]):
    mask = preds[i]
    mask = cv2.resize(mask, (1918, 1280), interpolation=cv2.INTER_LINEAR)
    mask = (mask > 0.95)
    mask = Image.fromarray((mask * 255).astype(np.uint8), mode='L')
    mask.save(pred_mask_path.format(ids_valid_split.values[i]))

## Validation

In [ ]:
def np_dice_value(y_true, y_pred):
    smooth = 1.
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

### Prediction

In [ ]:
run_name = 'unet-2017-09-03-1739'
model.load_weights('weights/{}.hdf5'.format(run_name))

val_imgs, val_masks = next(valid_generator(len(ids_valid_split)))
val_imgs = np.array(val_imgs)
val_masks = np.array(val_masks)
val_pred_masks = model.predict(val_imgs, batch_size=1)
masks_val_dices = [np_dice_value(mask, pred_mask) for (mask, pred_mask) in zip(val_masks, val_pred_masks)]

### Display the worst predicted mask for validation examples

In [ ]:
index = np.argsort(masks_val_dices)[7]
id = ids_valid_split.values[index]
utils.show_mask(train_path.format(id), val_masks[index].squeeze(), val_pred_masks[index].squeeze(), show_img=False)
print id, masks_val_dices[index]

In [ ]:
indices = np.argsort(masks_val_dices[masks_val_dices <= 99.6])
for id in indices:
        print(masks_val_dices[id])

### Histogram

In [ ]:
hist, bins = np.histogram(masks_val_dices, bins=50)
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
plt.bar(center, hist, align='center', width=width)
plt.show()

### Visualization

In [ ]:
indices = np.random.randint(len(ids_valid_split), size=3)
for index in indices:
    id = ids_valid_split.values[index]
    utils.show_mask(train_path.format(id), val_masks[index].squeeze(), val_pred_masks[index].squeeze(),
                    show_img=True, bbox = bboxes[id])

## Test

### Load Model

In [ ]:
# Create model first if required
run_name = 'unet-2017-08-20-5'
model.load_weights('weights/{}.hdf5'.format(run_name))

### Generate Submit

In [ ]:
batch_size = 16
threshold = 0.5
test_path = 'inputs/test1/' #'inputs/test/'
test_masks_path = 'outputs/test1_masks/' #None
generate_submit(model, input_size, batch_size, threshold, test_path, 'outputs/', run_name, test_masks_path)

### Visualization

In [ ]:
utils.show_test_masks(test_path, test_masks_path)